## Fake news Classifier using Bidirectional LSTM

In [1]:
import pandas as pd
df = pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.shape

(20800, 5)

In [6]:
df=df.dropna()

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
X= df.drop('label',axis=1)

In [9]:
y = df['label']

In [13]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.14.0'

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [20]:
voc_size = 5000

In [21]:
messages = X.copy()

In [22]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [26]:
import nltk
import re
from nltk.corpus import stopwords

In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ketan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
messages.reset_index(inplace=True)

In [35]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [36]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [37]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [38]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3395, 433, 2741, 962, 4888, 1913, 2599, 288, 31, 2242],
 [4716, 4628, 2863, 1894, 3579, 1537, 789],
 [899, 3924, 2950, 2031],
 [3354, 4045, 2139, 143, 115, 3898],
 [4457, 3579, 676, 3945, 1379, 405, 3579, 1424, 1287, 687],
 [4595,
  157,
  3958,
  4131,
  4524,
  1127,
  2298,
  2447,
  3218,
  4369,
  1728,
  2210,
  3040,
  202,
  789],
 [2652, 2505, 1340, 4405, 2372, 2422, 2815, 3867, 1496, 1323, 4363],
 [1864, 4757, 1433, 454, 4038, 4172, 1127, 1616, 1496, 1323, 4363],
 [2145, 2089, 4779, 2844, 1833, 955, 1770, 2138, 1127, 1201],
 [4268, 985, 2326, 3291, 2560, 3781, 1804, 3729],
 [1369, 3745, 415, 2467, 4665, 3361, 79, 2013, 347, 2594, 4730],
 [143, 2354, 4888, 955, 1127, 4038],
 [319, 3891, 4306, 503, 4193, 3840, 89, 696, 4610],
 [2380, 4701, 4743, 4129, 2905, 543, 962, 1496, 1323, 4363],
 [2379, 1977, 1460, 2095, 3238, 1496, 1323, 4363],
 [823, 2410, 4720, 1887, 986, 596, 2696, 102, 1800, 1603],
 [4006, 890, 4628],
 [4562, 4235, 2221, 4739, 1127, 2072, 3273, 789],
 [2385, 4301,

## Embedding Representation

In [45]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ...,  288,   31, 2242],
       [   0,    0,    0, ..., 3579, 1537,  789],
       [   0,    0,    0, ..., 3924, 2950, 2031],
       ...,
       [   0,    0,    0, ..., 1496, 1323, 4363],
       [   0,    0,    0, ..., 2391, 2745, 1130],
       [   0,    0,    0, ..., 3802, 2633,  321]])

In [46]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [47]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [48]:
### Train-test-split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [49]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 39s 50ms/step - loss: 0.2606 - accuracy: 0.8809 - val_loss: 0.1909 - val_accuracy: 0.9211
Epoch 2/10
383/383 [==============================] - 15s 38ms/step - loss: 0.1275 - accuracy: 0.9496 - val_loss: 0.2200 - val_accuracy: 0.9180
Epoch 3/10
383/383 [==============================] - 15s 39ms/step - loss: 0.0786 - accuracy: 0.9733 - val_loss: 0.2393 - val_accuracy: 0.9176
Epoch 4/10
383/383 [==============================] - 15s 39ms/step - loss: 0.0497 - accuracy: 0.9834 - val_loss: 0.3065 - val_accuracy: 0.9175
Epoch 5/10
383/383 [==============================] - 14s 37ms/step - loss: 0.0280 - accuracy: 0.9916 - val_loss: 0.3653 - val_accuracy: 0.9122
Epoch 6/10
383/383 [==============================] - 15s 38ms/step - loss: 0.0237 - accuracy: 0.9930 - val_loss: 0.4092 - val_accuracy: 0.9052
Epoch 7/10
383/383 [==============================] - 15s 38ms/step - loss: 0.0178 - accuracy: 0.9939 - val_loss: 0.4672 - val_accuracy:

In [50]:
y_pred=model.predict(X_test)

189/189 [==============================] - 5s 13ms/step


In [61]:
import numpy as np
y_pred=np.where(y_pred>=0.5,1,0)

In [63]:
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of y_pred: {y_pred.shape}")


Shape of y_test: (6035,)
Shape of y_pred: (5514,)


In [67]:
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score

# Ensure y_pred is a numpy array
y_pred = np.array(y_pred)

# Check the shapes
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of y_pred: {y_pred.shape}")

# Ensure y_pred has the same length as y_test
if y_test.shape != y_pred.shape:
    min_length = min(len(y_test), len(y_pred))
    y_test = y_test[:min_length]
    y_pred = y_pred[:min_length]
    print(f"Adjusted shape of y_test: {y_test.shape}")
    print(f"Adjusted shape of y_pred: {y_pred.shape}")

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


Shape of y_test: (5514,)
Shape of y_pred: (5514,)
Confusion Matrix:
[[1565 1561]
 [1193 1195]]


In [68]:
print(accuracy_score(y_test,y_pred))

0.500544069640914
